# Extracting Shapley DNN Semantics - EXPERIMENT

<img src = 'https://media.nationalgeographic.org/assets/photos/000/273/27302_c0-41-990-701_r1050x700.jpg?d4ccf3044d9da0d0118103be3a76bd1319370847' >

## Importing all necessary libraries


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import enum
import os
import platform
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import plotly.io as pio
import pickle
from scipy.stats import mannwhitneyu

### Setting random seed

In [ ]:
tf.random.set_seed(42)

## Define the possible nodes

In [ ]:
class Mode(enum.Enum):
    TRAIN = "TRAIN"
    TEST = "TEST"

## Define the current run-mode

In [ ]:
PC_NAME = platform.node().replace(" ", "_").replace(".", "_")
MODEL_DIR = f"../../output/titanic/models/experiment/{PC_NAME}"
PLOT_DIR = f"{MODEL_DIR}/plots"
MODE = Mode.TRAIN
if MODE == Mode.TEST and not os.path.exists(MODEL_DIR):
    print(f"Error: model not created (path \"{MODEL_DIR}\" does not exist), mode will be Mode.TRAIN")
    MODE = Mode.TRAIN

In [ ]:
def show_and_save_current_matplotlib_plt(plotname: str):
    if not os.path.exists(PLOT_DIR):
        os.makedirs(PLOT_DIR)
    file_n = f"{PLOT_DIR}/{plotname}"
    plt.savefig(file_n)
    plt.show()

def show_and_save_current_plotly_plt(plotname: str, fig, show=True):
    if not os.path.exists(PLOT_DIR):
        os.makedirs(PLOT_DIR)
    file_n = f"{PLOT_DIR}/{plotname}"
    if show:
        fig.show()
    pio.write_image(fig, file_n)

## Training the network

### Dropping PassengerId

In [ ]:
titanic_data = pd.read_csv('../../input/titanic/experiment/titanic-experiment-preprocessed.csv')
columns_to_drop = ['PassengerId']
titanic_data.drop(columns_to_drop, axis=1, inplace=True)

### Splitting train and test sets

In [ ]:
X_train = titanic_data.drop("Survived", axis=1)
Y_train = titanic_data["Survived"]

# Split data into 85% training and 15% testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X_train, Y_train, test_size=0.15, shuffle=True, random_state=999)

# Convert Y_train and Y_test to categorical
Y_train = to_categorical(np.array(Y_train), num_classes=2)
Y_test = to_categorical(np.array(Y_test), num_classes=2)

In [ ]:
n_inputs = len(X_train.columns)

## Defining model
Here, I have used different number of neurons for each layer and different value for dropout. You can play with these hyperparameter for better outut.

In [ ]:
def make_model():
    model = Sequential()
    model.add(Dense(units=16, input_shape=(n_inputs,), activation='relu'))
    model.add(Dense(units=32, activation='relu', kernel_initializer='he_normal', use_bias=False))
    model.add(Dense(units=2, activation='softmax'))
    return model


model = make_model()

## Model summary

In [ ]:
model.summary()

## Compiling and fitting model

In [ ]:
if MODE == Mode.TRAIN:
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])
    model.fit(X_train, Y_train, batch_size=16, verbose=2, epochs=1000)
    model.save(MODEL_DIR)
else:
    model.load_weights(MODEL_DIR)

### Prediction for test data

In [ ]:
from sklearn import metrics

Y_pred_rand = [0 if x[0] > x[1] else 1 for x in model.predict(X_test)]
Y_test_flat = [0 if x[0] > x[1] else 1 for x in Y_test]
print("*" * 10, ' TEST RESULTS ', "*" * 10)
print('Precision : ', np.round(metrics.precision_score(Y_test_flat, Y_pred_rand) * 100, 2))
print('Accuracy : ', np.round(metrics.accuracy_score(Y_test_flat, Y_pred_rand) * 100, 2))
print('Recall : ', np.round(metrics.recall_score(Y_test_flat, Y_pred_rand) * 100, 2))
print('F1 score : ', np.round(metrics.f1_score(Y_test_flat, Y_pred_rand) * 100, 2))
print('AUC : ', np.round(metrics.roc_auc_score(Y_test_flat, Y_pred_rand) * 100, 2))
print("*" * 10, ' TEST RESULTS ', "*" * 10)

In [ ]:
# plotting the confusion matrix in heatmap
matrix = metrics.confusion_matrix(Y_test_flat, Y_pred_rand)
sns.heatmap(matrix, annot=True, fmt='g')
show_and_save_current_matplotlib_plt("model_confusion_matrix.png")

# Co-Activation Graph

In [ ]:
x = X_test

# Compute intermediate layer activations
layer_names = [layer.name for layer in model.layers]
outputs = [model.get_layer(name).output for name in layer_names]
model_reduced = tf.keras.models.Model(inputs=model.inputs, outputs=outputs)
activations = model_reduced.predict(x)

# Compute model predictions on test data
predictions = [x for x in model.predict(x)]

# Concatenate predictions to activations
# Compute pairwise correlations between activations
activations_flat = np.concatenate([a.reshape(a.shape[0], -1) for a in activations], axis=1)
activations_flat = np.array([np.append(a, p) for a, p in zip(activations_flat, predictions)])
correlations = np.corrcoef(activations_flat, rowvar=False)

# Replace NaN values with 0 and remove corresponding rows and columns
correlations = np.nan_to_num(correlations)
nan_idx = np.where(np.isnan(correlations))[0]
if len(nan_idx) > 0:
    correlations = np.delete(correlations, nan_idx, axis=0)
    correlations = np.delete(correlations, nan_idx, axis=1)
    layer_names = [name for i, name in enumerate(layer_names) if i not in nan_idx]

# Plot co-activation matrix
plt.imshow(correlations, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title('Co-activation matrix')
plt.xticks(np.arange(len(layer_names)), layer_names, rotation=90)
plt.yticks(np.arange(len(layer_names)), layer_names)
show_and_save_current_matplotlib_plt("co_activation_matrix.png")


In [ ]:
import networkx as nx
import matplotlib.colors as mcolors
from networkx.algorithms.community import louvain_communities

threshold = 0.4

num_nodes = correlations.shape[0]
survived_output = num_nodes-1
died_output = num_nodes-2

# Define the node names as a list
# Create a dictionary that maps node indices to node names
node_labels = {i: (X_train.columns[i] if i < len(X_train.columns) else '') for i in range(num_nodes-2)}
node_labels.update({survived_output: "Survived"})
node_labels.update({died_output: "Died"})

min_v = 0.1
# Create the graph
G = nx.Graph()
for i in range(correlations.shape[0]):
    for j in range(i + 1, correlations.shape[1]):
        corr = min_v if (abs(correlations[i, j])) < min_v else correlations[i, j]
        G.add_edge(i, j, weight=corr)

# resolution 1.2 to slightly bias more cluster division, threshold 0.5 as research supports this
clusters = louvain_communities(G, weight='weight', resolution=1.2, threshold=0.5, seed=123)

# Draw the graph
fig, ax = plt.subplots(figsize=(10, 10))
pos = nx.spring_layout(G, weight='weight', k=1.3)
# nx.draw_networkx_nodes(G, pos, node_color='r', node_size=)
nx.draw_networkx_labels(G, pos, font_size=7, labels=node_labels, font_weight='bold')

edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
edges = list(edges)
weights = list(weights)

# this part will do the edge colorization
min_w, max_w = -1, 1
red = mcolors.hex2color('#FF0000')  # red color in RGB format
green = mcolors.hex2color('#00FF00')  # green color in RGB format
edge_colors = []
for w in weights:
    if abs(w) <= threshold:
        edge_colors.append('#00000000')
    else:
        # Map the weight value to a value between 0 and 1
        normalized_w = (w - min_w) / (max_w - min_w)

        # Use a linear interpolation to determine the color between red and green
        color = tuple((1 - normalized_w) * c1 + normalized_w * c2 for c1, c2 in zip(red, green))

        # Convert the color to hexadecimal format and append it to the list
        edge_colors.append(mcolors.rgb2hex(color))

# draw the network edges
edges, weights = zip(*[(edges[i], round(weights[i], 1)) for i in range(len(edges)) if abs(weights[i]) >= threshold])
nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=edge_colors, width=weights, edge_cmap=plt.cm.coolwarm)

# draw the louvain communities
colors = {}
for i, community in enumerate(clusters):
    color = plt.cm.Set1(i / len(clusters))
    colors.update({node: color for node in community})

for c in clusters:
    nx.draw_networkx_nodes(G, pos, nodelist=list(c), node_color=colors[list(c)[0]], node_size=1000)

plt.title(f'Co-Activation graph (edge correlation threshold=$abs(correlation)>={threshold}$)')
show_and_save_current_matplotlib_plt("co_activation_graph.png")

## SHAP (SHapley Additive exPlanations)

In [ ]:
import shap
import logging
import pickle
import os

recalc = False
logging.getLogger('shap').setLevel(logging.WARNING)

shap_values_full_model_fname = f"{MODEL_DIR}/shap_values_full_model.pkl"
shap_values_full_per_cluster_fname = f"{MODEL_DIR}/shap_values_per_cluster.pkl"
shap_values_full_per_feature_fname = f"{MODEL_DIR}/shap_values_per_feature.pkl"
shap_values_full_per_feature_dict_fname = f"{MODEL_DIR}/shap_values_per_feature_dict.pkl"
p_values_per_cluster_per_feature_fname = f"{MODEL_DIR}/p_values_per_cluster_per_feature.pkl"
significant_divergent_shap_distributions_fname = f"{MODEL_DIR}/significant_divergent_shap_distributions.pkl"

In [ ]:
def sample_size_full_model():
    return 500


def relative_node_sample_size(cluster_size):
    return sample_size_full_model() / cluster_size

### Full SHAP

In [ ]:
def do_or_load_and_write(func, file):
    if not recalc and os.path.exists(file):
        print(f'loading from file {file}')
        with open(file, 'rb') as f:
            return pickle.load(f)
    else:
        print(f'not loading from file {file}, running func and returning...')
        r = func()
        with open(file, 'wb') as f:
            pickle.dump(r, f)
        return r

In [ ]:
%%capture test

# This is the full SHAP explainer (begin layer to end layer)
def _2_class_to_binary_f(X, model):
    return np.array([0 if x[0] > x[1] else 1 for x in model.predict(X)])


def calc_full_shap():
    full_explainer = shap.Explainer(lambda x: _2_class_to_binary_f(x, model), X_test)
    sample_size = sample_size_full_model()
    print(f"Using full-model sample size {sample_size_full_model()}")
    return full_explainer(shap.utils.sample(X_test, sample_size))

In [ ]:
%%capture test
full_shap_values = do_or_load_and_write(calc_full_shap, shap_values_full_model_fname)

In [ ]:
def get_aggregated_model_shap(clazz, agg_func=lambda x: np.mean(np.abs(x))) -> dict:
    assert clazz in {0, 1}
    return {f: agg_func((full_shap_values.values[:, idx])) * (1 if clazz == 1 else -1) for idx, f in
            enumerate(X_train.columns)}


survived_mean_shap_vals = get_aggregated_model_shap(clazz=1)
died_mean_shap_vals = get_aggregated_model_shap(clazz=0)
full_model_shap_values = do_or_load_and_write(lambda: get_aggregated_model_shap(1, lambda x: x), shap_values_full_per_feature_dict_fname)

In [ ]:
full_model_shap_values

In [ ]:
shap.plots.beeswarm(full_shap_values)

In [ ]:
plt.bar(died_mean_shap_vals.keys(), died_mean_shap_vals.values())
plt.xlabel('Feature')
plt.ylabel('SHAP value')
plt.xticks(rotation=90)
plt.title('SHAP values per feature on class 0 ("died")')
show_and_save_current_matplotlib_plt("model_shap_values_bar_chart_died.png")

plt.bar(survived_mean_shap_vals.keys(), survived_mean_shap_vals.values())
plt.xlabel('Feature')
plt.ylabel('SHAP value')
plt.xticks(rotation=90)
plt.title('SHAP values per feature on class 1 ("survived")')
show_and_save_current_matplotlib_plt("model_shap_values_bar_chart_survived.png")

### Clustered SHAP

In [ ]:
named_clusters = {f"cluster_{i}": c for i, c in enumerate(clusters)}
print(named_clusters)

#### Steps new SHAP-research idea
1. Per node, per instance, activation SHAP values (per instance)
2. Per cluster, per feature, SHAP distribution
3. General SHAP values per feature on entire model
4. Check where general SHAP values per feature lay within the distribution per feature per cluster

In [ ]:
nodes = set()
for c in clusters:
    for node in c:
        nodes.add(node)
print(f"Number of nodes: {len(nodes)}")

In [ ]:
model_function_per_node = {}
node_num = 0
for layer_id, layer in enumerate(model.layers):
    m = tf.keras.Model(inputs=model.input, outputs=layer.output)
    for i in range(layer.output.shape[1]):
        if node_num in nodes:
            model_function_per_node[node_num] = m
        node_num += 1

In [ ]:
def node_activation_f(X, node: int):
    if node in model_function_per_node:
        m = model_function_per_node[node]
        local_node_num = node % m.output.shape[1]
        return np.array([x[local_node_num] for x in m.predict(X)])
    raise Exception(f"{n} not in {model_function_per_node.keys()}")

In [ ]:
%%capture test
nodes = [n for n in model_function_per_node.keys()]

clusters_size_per_node = {}
for c in clusters:
    for n in c:
        clusters_size_per_node[n] = len(c)

In [ ]:
def calc_shap_per_feature():
    shap_values_per_feature = {x: {n: [] for n in nodes} for x in X_train.columns}

    current_node_number = nodes[0]
    ex = shap.Explainer(lambda x: node_activation_f(x, current_node_number), X_train)
    for n in nodes:
        sample_size = int(np.ceil(relative_node_sample_size(clusters_size_per_node[n])))
        sample = shap.utils.sample(X_test, sample_size)
        current_node_number = n
        vals = ex(sample).values
        for svarr in vals:
            for idx, sv in enumerate(svarr, 0):
                shap_values_per_feature[X_train.columns[idx]][n].append(sv)
    return shap_values_per_feature

In [ ]:
%%capture test
shap_values_per_feature = do_or_load_and_write(calc_shap_per_feature, shap_values_full_per_feature_fname)

In [ ]:
def calc_shap_per_cluster():
    shap_values_per_cluster = {c: {f: [] for f in X_train.columns} for c in named_clusters.keys()}
    for feature, n_sv in shap_values_per_feature.items():
        for node, sv in n_sv.items():
            for c_name, nodes in named_clusters.items():
                if node in nodes:
                    for _sv in sv:
                        shap_values_per_cluster[c_name][feature].append(_sv)
    return shap_values_per_cluster

In [ ]:
shap_values_per_cluster = do_or_load_and_write(calc_shap_per_cluster, shap_values_full_per_cluster_fname)

In [ ]:
import plotly.graph_objects as go

#real deal
n_bins = 50
y_range = [0,0.15]
x_range = [-2,2]
normalise = 'probability'

for cluster_name, feats_with_shaps  in shap_values_per_cluster.items():

    for feat_name, shap_values in feats_with_shaps.items():

        #main object
        fig = go.Figure()

        #first data (cluster specific)
        fig.add_trace(go.Histogram(
            x=shap_values, #shaps of feature of cluster
            histnorm=normalise,
            name=f'{feat_name} of {cluster_name}',
            nbinsx=n_bins,
            # marker_color='blue',
        ))

        #data for second plot: shaps of feature of the whole model
        fig.add_trace(go.Histogram(
            x=full_model_shap_values[feat_name],
            histnorm=normalise,
            name=f'{feat_name} of the actual model',
            nbinsx=n_bins,
            # marker_color='#EB89B5',
        ))


        #to tweak appearances
        fig.update_layout(barmode='overlay',
            title_text=f'Distributions of {feat_name} in {cluster_name} against whole model', # title of plot
            xaxis_title_text='Shapley value', # xaxis label
            yaxis_title_text='percentage', # yaxis label

            yaxis=dict(
                range=y_range
            ),
            xaxis=dict(
                range=x_range
            ),
        )

        # Reduce opacity to see both histograms
        fig.update_traces(opacity=0.7)
        show_and_save_current_plotly_plt(f'histg_{(feat_name.lower())}_{cluster_name}.png', fig, show=False)

In [ ]:
import plotly.graph_objects as go

#real deal
n_bins = 50
y_range = [0,0.15]
x_range = [-2,2]
normalise = 'probability'

shap_values_per_feature_per_cluster  = {}

for clus,sfc_dict in shap_values_per_cluster.items():
    for feat, sv in sfc_dict.items():
        if feat in shap_values_per_feature_per_cluster:
            shap_values_per_feature_per_cluster[feat][clus] = sv
        else:
            shap_values_per_feature_per_cluster[feat] = {}
            shap_values_per_feature_per_cluster[feat][clus] = sv

for feat, sv_cluster  in shap_values_per_feature_per_cluster.items():

        #main object
        fig = go.Figure()

        for clus, sv in sv_cluster.items():
            #first data (cluster specific)
            fig.add_trace(go.Histogram(
                x=sv, #shaps of feature of cluster
                histnorm=normalise,
                name=f'{feat} of {clus}',
                nbinsx=n_bins,
                # marker_color='blue',
            ))

        #data for second plot: shaps of feature of the whole model
        fig.add_trace(go.Histogram(
            x=full_model_shap_values[feat],
            histnorm=normalise,
            name=f'{feat} of the actual model',
            nbinsx=n_bins,
            # marker_color='#EB89B5',
        ))


        #to tweak appearances
        fig.update_layout(barmode='overlay',
            title_text=f'Distributions of {feat} in all clusters against whole model', # title of plot
            xaxis_title_text='Shapley value', # xaxis label
            yaxis_title_text='percentage', # yaxis label

            yaxis=dict(
                range=y_range
            ),
            xaxis=dict(
                range=x_range
            ),
        )

        # Reduce opacity to see both histograms
        fig.update_traces(opacity=0.7)
        show_and_save_current_plotly_plt(f'histg_{(feat.lower())}_clusters_combined.png', fig, show=False)

### Finding significant different distributions

In [ ]:
#adjust for correct model
shaps_full_model = full_model_shap_values
shaps_per_cluster = shap_values_per_cluster
shaps_per_feature = shap_values_per_feature_per_cluster

def find_pvalues_per_feature(shaps_full_model, dict_of_features_with_shaps):
    '''
    Calc mann-withney for every feature against full_model

    Returns: dict with p-values per feature
    '''
    dict_of_pvalues_per_feature = {}

    #loop over available features
    for feature_name, shapley_values in dict_of_features_with_shaps.items():

        #calc M-W two sided (same or different)
        _ , p_value = mannwhitneyu(shaps_full_model[feature_name], shapley_values, alternative='two-sided')

        dict_of_pvalues_per_feature[feature_name] = round(p_value,4)

    return dict_of_pvalues_per_feature


def find_p_values_per_cluster_per_feature():
    p_values_per_cluster_per_feature = {}

    #loop over every cluster
    for cluster_name in shaps_per_cluster.keys():
        result = find_pvalues_per_feature(shaps_full_model=shaps_full_model,
                                          dict_of_features_with_shaps=shaps_per_cluster[cluster_name])
        p_values_per_cluster_per_feature[cluster_name] = result

    return p_values_per_cluster_per_feature


In [ ]:
p_values_per_cluster_per_feature = do_or_load_and_write(find_p_values_per_cluster_per_feature, p_values_per_cluster_per_feature_fname)

In [ ]:
print(p_values_per_cluster_per_feature)

In [ ]:
def filter_significant_differences(p_values_per_cluster_per_feature, signicance_level):
    significance_reporter_per_cluster = {cluster_name:[] for cluster_name in p_values_per_cluster_per_feature.keys()}

    for cluster_name, dict_of_pvalues_per_feature in p_values_per_cluster_per_feature.items():
        for feature_name, p_value in dict_of_pvalues_per_feature.items():
            if p_value <= signicance_level:
                significance_reporter_per_cluster[cluster_name].append(feature_name)
    return significance_reporter_per_cluster

In [ ]:
filter_significant_differences_result = do_or_load_and_write(lambda:filter_significant_differences(p_values_per_cluster_per_feature, 0.05), significant_divergent_shap_distributions_fname)

In [ ]:
filter_significant_differences_result